In [ ]:
using TensorOperations, TensorKit
import TensorKit.×
using FLoops
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./models/Hubbard_Z2SU2.jl")
# include("./models/tJ_Z2.jl")
@show pkgversion(TensorKit)
using JLD2, HDF5
using Strided
Strided.enable_threads()

In [ ]:
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8.0
para[:μ] = 4.0
para[:pspace] = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
Hij = get_op_Hubbard("hij", para)
@show Hij

Hijp = permute(Hij, ((2, 1), (4, 3)))
@show Hijp
@assert Hij == Hijp
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./models/tJ_Z2.jl")
include("./simple_update_Fermionic/simple_update.jl")
include("./fast_full_update_Fermionic/fast_full_update.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")

ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)

Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, ["Δₛ", "Δₛdag"], para; site1=[1, 1], site2=[0, 2], get_op=get_op_tJ)

nothing

In [ ]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")


In [1]:
using TensorKit
using Zygote
using ChainRulesCore
import TensorKit.×
using FLoops
import Zygote.Buffer

include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./models/tJ_Z2.jl")
include("./simple_update_Fermionic/simple_update.jl")
include("./fast_full_update_Fermionic/fast_full_update.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./AD/ADutils.jl")

para = Dict{Symbol,Any}()
para[:t] = 3.0
para[:tp] = 0.51
para[:J] = 1.0
para[:Jp] = 0.0289
para[:h] = 0.6
para[:μ] = 5.4  # set μ = 5.2,  n = 0.89986
para[:τlisSU] = [1.0, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0001]
para[:τlisFFU] = [0.01, 0.005, 0.001, 0.0001]
para[:minStep1τ] = 50   # 对每个虚时步长 τ , 最少投影这么多步
para[:maxStep1τ] = 1000  # 对每个虚时步长 τ , 最多投影这么多步
para[:maxiterFFU] = 60
para[:tolFFU] = 1e-10  # FFU 中损失函数的 Tolerence
para[:Dk] = 6  # Dkept in the simple udate
para[:χ] = 100  # env bond dimension
para[:CTMit] = 30  # Maximum CTMRG iteration times
para[:CTMparallel] = true  # use parallel CTMRG or not. Use with MKL.
para[:CTMthreshold] = 1e-12
para[:Etol] = 1e-6  # simple update 能量差小于 para[:Etol]*τ² 这个数就可以继续增大步长. 1e-5对小size
para[:verbose] = 1
para[:TrotterOrder] = 2 # 用几阶Trotter分解,设为1或2
para[:pspace] = Rep[ℤ₂](0 => 1, 1 => 2)
get_op = get_op_tJ

pspace = Rep[ℤ₂](0 => 1, 1 => 2)
aspacelr = Rep[ℤ₂](0 => 1, 1 => 2)
aspacetb = Rep[ℤ₂](0 => 1, 1 => 2)
Lx = 2
Ly = 2

ipeps = iPEPS(iPEPSΓΛ(pspace, aspacelr, aspacetb, Lx, Ly; dtype=Float64))
envs = iPEPSenv(ipeps)
ipepsbar = bar(ipeps)
nothing

In [ ]:
E, grad = withgradient(ipeps, envs) do peps, env
    Cal_Energy(peps, env, get_op_tJ, para)
end
∂ε_∂A, ∂ε_∂e = grad

@show E
@show ∂ε_∂A.Lx
@show typeof(∂ε_∂A)
nothing

In [2]:
using ChainRulesTestUtils
include("./AD/ADutils.jl")

test_rrule(_2siteObs_adjSite, ipeps, ipepsbar, envs, ["hijNN"], para, [1, 1], [1, 2], get_op; ADflag=true)

test_rrule: _2siteObs_adjSite on iPEPS,iPEPS,iPEPSenv,Vector{String},Dict{Symbol, Any},Vector{Int64},Vector{Int64},typeof(get_op_tJ): Error During Test at C:\Users\quxin\.julia\packages\ChainRulesTestUtils\Ko1Wr\src\testers.jl:202
  Got exception outside of a @test
  ArgumentError: Tangent for the primal TensorKit.SortedVectorDict{Z2Irrep, Matrix{Float64}} should be backed by a AbstractDict type, not by NamedTuple{(:keys, :values), Tuple{NoTangent, Vector{Matrix{Float64}}}}.
  Stacktrace:
    [1] _backing_error(P::Type, G::Type, E::Type)
      @ ChainRulesCore C:\Users\quxin\.julia\packages\ChainRulesCore\I1EbV\src\tangent_types\structural_tangent.jl:349
    [2] Tangent{TensorKit.SortedVectorDict{Z2Irrep, Matrix{Float64}}, NamedTuple{(:keys, :values), Tuple{NoTangent, Vector{Matrix{Float64}}}}}(backing::NamedTuple{(:keys, :values), Tuple{NoTangent, Vector{Matrix{Float64}}}})
      @ ChainRulesCore C:\Users\quxin\.julia\packages\ChainRulesCore\I1EbV\src\tangent_types\structural_tangent.

Test.TestSetException: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.

In [ ]:
function f(A::TensorMap, B::TensorMap)
    D = isometry(space(A)[2], space(A)[2])
    # D = id(space(A)[2])
   return @tensoropt C = A[a, b] * D'[c, b] * B[c, a]
end

pspace2 = Rep[ℤ₂](0 => 2, 1 => 2)
A = TensorMap(randn, Float64, pspace, pspace2)
B = TensorMap(randn, Float64, pspace2, pspace)
nothing

In [ ]:
grd = gradient(f, A, B)
@show grd
@show typeof(grd[1])
nothing

In [ ]:
using TensorKit
using ChainRulesCore
using Zygote

struct Foo
    A::Matrix{AbstractTensorMap}
    c::Int
end
function foo_mul(foo::Foo, b::TensorMap)
    @tensor C[x; z] := foo.A[1, 1][x, y] * b[y, z]
    return C
end
function ChainRulesCore.rrule(::typeof(foo_mul), foo::Foo, b::TensorMap)
    y = foo_mul(foo, b)
    function foo_mul_pullback(ȳ)
        f̄ = NoTangent()
        f̄oo = Foo(ȳ * b', ZeroTangent())
        b̄ = foo.A' * ȳ
        return f̄, f̄oo, b̄
    end
    return y, foo_mul_pullback
end

In [ ]:
pspace2 = Rep[ℤ₂](0 => 2, 1 => 2)
pspace1 = Rep[ℤ₂](0 => 1, 1 => 1)
x = TensorMap(randn, Float64, pspace1, pspace2)
y = TensorMap(randn, Float64, pspace2, pspace1)
mat = [x x; x x]
val, jac = withjacobian(Foo(mat, 3), y) do x, y
    foo_mul(x, y)
end
@show jac

val, back = pullback(foo_mul, Foo(mat, 3), y)
@show val
xp = TensorMap(randn, Float64, pspace2, pspace1)
@show back(xp)
nothing

In [ ]:
using Zygote
import Zygote.Buffer

# Define your struct
struct iPEPSenv
    field1::Union{Matrix,Buffer}
    field2::Union{Matrix,Buffer}
    # Add more fields as necessary
end

# Function to convert fields of iPEPSenv to Zygote.Buffer
function Zygote.Buffer(env::iPEPSenv)
    buffers = iPEPSenv(
        Zygote.Buffer(env.field1),  # Convert each AbstractArray field to a Buffer
        Zygote.Buffer(env.field2)   # Do this for all AbstractArray fields
        # Add more fields as necessary
    )
    return buffers
end

# Example usage
env = iPEPSenv(rand(3, 3), rand(4, 4))  # Example struct initialization
buffered_env = Zygote.Buffer(env)  # Convert to Zygote.Buffer

println(buffered_env)  # Check the result


In [ ]:
using OptimKit
using Zygote

# Define the objective function
function objective(x)
    return (x[1] - 1)^2 + (x[2] - 2)^2, Zygote.gradient(objective, x)[1]
end

# Initialize the starting point
x0 = [0.0, 0.0]

# Create an L-BFGS optimizer
optimizer = LBFGS(2)

# Run the optimization
result = OptimKit.optimize(objective, x0, optimizer)

# Display the result
@show result
